In [1]:
import yaml

import torch
import pandas as pd

from xopt import Xopt
from lume_model.utils import variables_from_yaml
from lume_model.torch import PyTorchModel, LUMEModule

## Load Data from BO and BE Runs

In [2]:
data_dir = "../../data/"

bo_files = [
    "../../optimization/optimization_results.yml",
    data_dir + "20230818_LCLS_optimize/nn_optimization.yml",
    data_dir + "20230821_LCLS_Injector/normal_optimization.yml",
    data_dir + "20230821_LCLS_Injector/nn_optimization.yml",
    data_dir + "20230821_LCLS_Injector/turbo_optimization.yml",
    data_dir + "20230821_LCLS_Injector/turbo_optimization_2.yml",
    data_dir + "20230821_LCLS_Injector/turbo_optimization_3.yml",

    data_dir + "20230825_LCLS_Injector/nn_optimization_vanilla.yml",
    data_dir + "20230825_LCLS_Injector/nn_optimization.yml",
    data_dir + "20230825_LCLS_Injector/nn_optimization_1.yml",
    data_dir + "20230825_LCLS_Injector/nn_optimization_cal_high_reg.yml",
    data_dir + "20230825_LCLS_Injector/nn_optimization_cal_high_reg_adj_bounds.yml",
    data_dir + "20230825_LCLS_Injector/nn_optimization_cal_low_reg_adj_bounds.yml",
    data_dir + "20230825_LCLS_Injector/nn_optimization_cal_low_reg_adj_bounds_1.yml",
    data_dir + "20230825_LCLS_Injector/nn_optimization_cal_low_reg_constraint_prior.yml",    
]

be_files = [
    "../../optimization/be_results.yml",
    data_dir + "20230821_LCLS_Injector/BE_2.yml",
]

In [3]:
# BO data
dfs_bo, vocs_bo = [], []
for filename in bo_files:
    with open(filename) as f:
        d = yaml.safe_load(f)
    dfs_bo.append(pd.DataFrame(d["data"]).dropna())
    vocs_bo.append(d["vocs"])
df_bo = pd.concat(dfs_bo)
len(df_bo)

834

In [4]:
# BE data
dfs_be, vocs_be = [], []
for filename in be_files:
    with open(filename) as f:
        d = yaml.safe_load(f)
    dfs_be.append(pd.DataFrame(d["data"]).dropna())
    vocs_be.append(d["vocs"])
df_be = pd.concat(dfs_be)
len(df_be)

137

In [5]:
# check vocs consistency
vocs = vocs_bo[0]
all([v["variables"].keys() == vocs["variables"].keys() for v in vocs_bo + vocs_be])

True

In [6]:
# full data set
df = pd.concat([df_bo, df_be])

## Load NN Prior Model

In [7]:
model_path = "../lcls_cu_injector_nn_model/"

# load sim_to_nn transformers
input_sim_to_nn = torch.load(model_path + "model/input_sim_to_nn.pt")
output_sim_to_nn = torch.load(model_path + "model/output_sim_to_nn.pt")

# load pv_to_sim transformers
input_pv_to_sim = torch.load(model_path + "model/input_pv_to_sim.pt")
output_pv_to_sim = torch.load(model_path + "model/output_pv_to_sim.pt")

# load in- and output variable specification
input_variables, output_variables = variables_from_yaml(open(model_path + "model/pv_variables.yml"))

# replace keys in input variables
input_variables = {name.replace("BACT", "BCTRL"): ele for name, ele in input_variables.items()}

# create LUME-model
lume_model = PyTorchModel(
    model_file=model_path + "model/model.pt",
    input_variables=input_variables,
    output_variables=output_variables,
    input_transformers=[input_pv_to_sim, input_sim_to_nn],
    output_transformers=[output_sim_to_nn, output_pv_to_sim],
)

# wrap in LUMEModule
lume_module = LUMEModule(
    model=lume_model,
    feature_order=list(vocs["variables"].keys()),
    output_order=lume_model.outputs[0:2],
)

# define objective model
class ObjectiveModel(torch.nn.Module):
    def __init__(self, model: LUMEModule):
        super(ObjectiveModel, self).__init__()
        self.model = model

    @staticmethod
    def function(sigma_x: torch.Tensor, sigma_y: torch.Tensor) -> torch.Tensor:
        # using this calculation due to occasional negative values
        # return torch.sqrt(torch.abs(sigma_x) * torch.abs(sigma_y))
        return torch.sqrt(sigma_x ** 2 + sigma_y ** 2)

    def forward(self, x) -> torch.Tensor:
        idx_sigma_x = self.model.output_order.index("OTRS:IN20:571:XRMS")
        idx_sigma_y = self.model.output_order.index("OTRS:IN20:571:YRMS")
        sigma_x = self.model(x)[..., idx_sigma_x]
        sigma_y = self.model(x)[..., idx_sigma_y]
        return self.function(sigma_x, sigma_y)


objective_model = ObjectiveModel(lume_module)

## Load Calibrated NN Prior Models

In [8]:
calibration_path = "../../nn_prior/calibration/"

regs, cal_models = ["low", "high"], []
for reg in regs:
    # load nn_to_cal transformers
    input_nn_to_cal = torch.load(calibration_path + f"input_nn_to_cal_{reg}_reg.pt")
    output_nn_to_cal = torch.load(calibration_path + f"output_nn_to_cal_{reg}_reg.pt")
    
    # create LUME-model
    cal_lume_model = PyTorchModel(
        model_file=model_path + "model/model.pt",
        input_variables=input_variables,
        output_variables=output_variables,
        input_transformers=[input_pv_to_sim, input_sim_to_nn, input_nn_to_cal],
        output_transformers=[output_nn_to_cal, output_sim_to_nn, output_pv_to_sim],
    )
    
    # wrap in LUMEModule
    cal_lume_module = LUMEModule(
        model=cal_lume_model,
        feature_order=list(vocs["variables"].keys()),
        output_order=cal_lume_model.outputs[0:2],
    )
    
    cal_models.append(ObjectiveModel(cal_lume_module))

## Calculate Correlations and MAEs

In [9]:
# define calculation of correlation
def calc_corr(a: torch.Tensor, b: torch.Tensor) -> torch.Tensor:
    corr = torch.corrcoef(torch.stack([a.squeeze(), b.squeeze()]))
    return corr[0, 1]

In [10]:
# define calculation of MAE
def calc_mae(a: torch.Tensor, b: torch.Tensor) -> torch.Tensor:
    return torch.nn.functional.l1_loss(a, b)

In [11]:
metrics = pd.DataFrame(columns=[["All Data"] * 2 + ["BO"] * 2 + ["BE"] * 2, ["Corr", "MAE"] * 3])
for model in [objective_model] + cal_models:
    model_metrics = []
    for data in [df, df_bo, df_be]:
        
        # inputs and targets
        x = torch.from_numpy(data[lume_module.feature_order].to_numpy())
        y = torch.from_numpy(data[list(vocs["objectives"].keys())[0]].to_numpy())
        
        # predictions
        pred = model(x)
        
        # model metrics
        model_metrics.append("{:.2f}".format(calc_corr(pred, y).item()))
        model_metrics.append("{:.2f}".format(calc_mae(pred, y).item()))

    metrics.loc[len(metrics.index)] = model_metrics
metrics.index = ["Original Model", "Calibrated (Low Reg)", "Calibrated (High Reg)"]

In [12]:
metrics

All Data            BO            BE        
                          Corr     MAE  Corr     MAE  Corr     MAE
Original Model            0.12  372.78  0.09  367.49  0.25  404.98
Calibrated (Low Reg)      0.21  610.75  0.21  638.24  0.53  443.41
Calibrated (High Reg)     0.19  262.97  0.18  273.55  0.43  198.55